#BRAND LOGO DETECTION

##IMPORTING LIBRARIES

In [1]:
%%capture
!pip install tensorflow-addons

In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow_addons.metrics import F1Score
from sklearn.metrics import accuracy_score

In [9]:
BATCH_SIZE = 8
HEIGHT,WIDTH = 224,224
CHANNELS = 3
NUM_CLASSES =6 
SEED = 143

In [ ]:

# labels={'Burger King': 0, 'KFC': 1,'McDonalds': 2,'Other': 3,'Starbucks': 4,'Subway': 5}

##LOADING DATASET

In [2]:
!wget https://huggingface.co/spaces/Sa-m/Flask-App-test-purpose/resolve/main/pot_hole_data/brand_logo_data.zip


--2022-09-19 14:40:23--  https://huggingface.co/spaces/Sa-m/Flask-App-test-purpose/resolve/main/pot_hole_data/brand_logo_data.zip
Resolving huggingface.co (huggingface.co)... 52.6.16.131, 52.202.207.64, 2600:1f18:147f:e850:db35:e0c7:187b:c770, ...
Connecting to huggingface.co (huggingface.co)|52.6.16.131|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/f1/ec/f1ec6a79a1cb501b664cae8c6297c6f85af0f72549a56fe5a73b0a79c854af8d/546ba525f4441b7cf7fd596f411c6b0cda505fac7de47aee078ab23f4b4659d0?response-content-disposition=attachment%3B%20filename%3D%22brand_logo_data.zip%22 [following]
--2022-09-19 14:40:24--  https://cdn-lfs.huggingface.co/repos/f1/ec/f1ec6a79a1cb501b664cae8c6297c6f85af0f72549a56fe5a73b0a79c854af8d/546ba525f4441b7cf7fd596f411c6b0cda505fac7de47aee078ab23f4b4659d0?response-content-disposition=attachment%3B%20filename%3D%22brand_logo_data.zip%22
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 13.227.

In [3]:
%%capture
!unzip /content/brand_logo_data.zip

##Download the video file

In [ ]:
# # https://drive.google.com/file/d/1z1_Bgthqy2RaU8Yvdcscqul4DbQdoAFD/view?usp=sharing
# # https://drive.google.com/file/d/1KwK5YdfaYJQT9CINhLa8g9oQtZawUCOP/view?usp=sharing
# # https://drive.google.com/file/d/1f0UrrqcAMyZk4WjQFt7l1W0z_O0-LTqt/view?usp=sharing

# download('13gX_hg3903OKBviXVur5i7VycOlW-0nW','./content/test_video/test_vid.mp4',False)

##VIDEO TO FRAMES

In [ ]:
import cv2


def vid_to_frames(video_path):
    
  # Read the video from specified path
  cam = cv2.VideoCapture(video_path)
    #../content/content/test_vid.mp4 
  try:
        
      # creating a folder named data
      if not os.path.exists('test/test_data'):
          os.makedirs('test/test_data')

    
  # if not created then raise error
  except OSError:
      print ('Error: Creating directory of data')
    
  
  currentframe = 0
  os.chdir('/content/test/test_data')  
  while(True):
        
      
      ret,frame = cam.read()
    
      if ret:
          
          name = '../test_data/frame' + str(currentframe) + '.jpg'
          cv2.imwrite(name, frame)
          currentframe += 1
      else:
          break
  cam.release()
  cv2.destroyAllWindows()
  %cd /content/


In [ ]:
vid_to_frames('../content/content/test_video/test_vid.mp4')

/content


### local test set

In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range = 0.2, 
    zoom_range = 0.2,
   )
my_test = test_datagen.flow_from_directory(
                             '/content/test',
                             target_size = (HEIGHT,WIDTH),
                             batch_size = BATCH_SIZE,
                             class_mode = "categorical",
                             shuffle = False,
                             seed = SEED,
                            #  classes=CLASSES,
                             color_mode='rgb'
                            )

Found 1500 images belonging to 1 classes.


##TRAIN DATASET PREPROCESSING

In [10]:
TRAIN_PATH = "../content/logos3/train"
TEST_PATH = "../content/logos3/test"

print(os.listdir(TRAIN_PATH))


['Subway', 'Burger King', 'Other', 'KFC', 'McDonalds', 'Starbucks']


In [11]:

CLASSES= os.listdir(TRAIN_PATH)
CLASSES.sort()
# ['Burger King', 'Subway', 'McDonalds', 'Starbucks', 'Other', 'KFC']

In [12]:
CLASSES

['Burger King', 'KFC', 'McDonalds', 'Other', 'Starbucks', 'Subway']

In [13]:
# from tensorflow.keras.applications.densenet import preprocess_input 

In [14]:
#Data generated
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range = 0.2, 
    zoom_range = 0.2,
    validation_split=0.22,  
    # preprocessing_function= tf.keras.applications.nasnet.preprocess_input
   )

#Data stored in train & test 
train_ds = train_datagen.flow_from_directory(
                             TRAIN_PATH,
                             target_size = (HEIGHT,WIDTH),
                             batch_size = BATCH_SIZE,
                             class_mode = "categorical",
                             shuffle = True,
                             seed = SEED,
                             classes=CLASSES,
                             subset = "training",
                             color_mode='rgb'
                            )
val_ds = train_datagen.flow_from_directory(
                             TRAIN_PATH,
                             target_size = (HEIGHT,WIDTH),
                             batch_size = BATCH_SIZE,
                             class_mode = "categorical",
                             shuffle = False,
                             seed = SEED,
                             classes=CLASSES,
                             subset = "validation",
                             color_mode='rgb'
                            )


classes_dict = train_ds.class_indices
# classes_dict

Found 1358 images belonging to 6 classes.
Found 380 images belonging to 6 classes.


In [15]:
train_ds.class_indices

{'Burger King': 0,
 'KFC': 1,
 'McDonalds': 2,
 'Other': 3,
 'Starbucks': 4,
 'Subway': 5}

In [16]:
# labels=train_ds.class_indices

##TEST DATASET PREPROCESSING

In [17]:

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range = 0.2, 
    zoom_range = 0.2,
    # preprocessing_function=tf.keras.applications.nasnet.preprocess_input
   )
test_ds = test_datagen.flow_from_directory(
                             TEST_PATH,
                             target_size = (HEIGHT,WIDTH),
                             batch_size = BATCH_SIZE,
                             class_mode = "categorical",
                             shuffle = False,
                             seed = SEED,
                             classes=CLASSES,
                             color_mode='rgb'
                            )

Found 560 images belonging to 6 classes.


## VGG16

In [ ]:
def create_model1():
    model = tf.keras.applications.VGG16(weights= "imagenet",
                                    include_top=False,
                                    input_shape=(HEIGHT,WIDTH,CHANNELS), pooling="avg")
    predictions = tf.keras.layers.Dense(32, activation='relu', name='predictions')(model.output)
    model = tf.keras.Model(inputs=model.input, outputs=predictions)
    
    
    model = tf.keras.Model(model.input, model.layers[-2].output)
    
    x = tf.keras.layers.Dense(512, activation = "relu")(model.output)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(256, activation = "relu")(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(128, activation = "relu")(x)
    # x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(64, activation = "relu")(x)
    outputs = tf.keras.layers.Dense(NUM_CLASSES, activation = "softmax", dtype = tf.float32)(x)
    model = tf.keras.Model(model.input,outputs)
    
    for layer in model.layers[:-10]:
        layer.trainble = False
    return model


model=create_model1()  

58900480/58889256 [==============================] - 0s 0us/step


## InceptionResNetV2

In [ ]:

def create_model2():
    model = tf.keras.applications.InceptionResNetV2(weights= "imagenet",
                                    include_top=False,
                                    input_shape=(HEIGHT,WIDTH,CHANNELS), pooling="avg")
    predictions = tf.keras.layers.Dense(32, activation='relu', name='predictions')(model.output)
    model = tf.keras.Model(inputs=model.input, outputs=predictions)
    
    
    model = tf.keras.Model(model.input, model.layers[-2].output)
    
    x = tf.keras.layers.Dense(512, activation = "relu")(model.output)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(256, activation = "relu")(x)
    # x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(128, activation = "relu")(x)
    # x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(64, activation = "relu")(x)
    outputs = tf.keras.layers.Dense(6, activation = "softmax", dtype = tf.float32)(x)
    model = tf.keras.Model(model.input,outputs)
    
    for layer in model.layers[:-2]:
        layer.trainble = False
    return model


model=create_model2()  

219062272/219055592 [==============================] - 7s 0us/step


## NASNET

In [ ]:

def create_model3():
    model = tf.keras.applications.NASNetMobile(weights= "imagenet",
                                    include_top=False,
                                    input_shape=(HEIGHT,WIDTH,CHANNELS), pooling="avg")
    predictions = tf.keras.layers.Dense(32, activation='relu', name='predictions')(model.output)
    model = tf.keras.Model(inputs=model.input, outputs=predictions)
    

    model = tf.keras.Model(model.input, model.layers[-2].output)
    
    x = tf.keras.layers.Dense(256, activation = "relu")(model.output)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(256, activation = "relu")(x)
    x = tf.keras.layers.Dense(128, activation = "relu")(x)
    x = tf.keras.layers.Dense(64, activation = "relu")(x)
    outputs = tf.keras.layers.Dense(NUM_CLASSES, activation = "softmax", dtype = tf.float32)(x)
    model = tf.keras.Model(model.input,outputs)
    
    for layer in model.layers[:-4]:
        layer.trainble = False
    return model

model = create_model3()


20004864/19993432 [==============================] - 0s 0us/step


## BASE MODEL

In [ ]:

IMAGE_SIZE=(224,224)
def conv_block(filters, inputs):
    x = tf.keras.layers.SeparableConv2D(filters, 3, activation="relu", padding="same")(inputs)
    x = tf.keras.layers.SeparableConv2D(filters, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    outputs = tf.keras.layers.MaxPool2D()(x)
    return outputs


def dense_block(units, dropout_rate, inputs):
    x = tf.keras.layers.Dense(units, activation="relu")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    outputs = tf.keras.layers.Dropout(dropout_rate)(x)

    return outputs
def build_model():
    inputs = tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    x = tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)
    x = tf.keras.layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.MaxPool2D()(x)

    x = conv_block(32, x)
    x = conv_block(64, x)

    x = conv_block(128, x)
    x = tf.keras.layers.Dropout(0.2)(x)

    x = conv_block(256, x)
    x = tf.keras.layers.Dropout(0.2)(x)

    x = tf.keras.layers.Flatten()(x)
    x = dense_block(512, 0.7, x)
    x = dense_block(128, 0.5, x)
    x = dense_block(64, 0.3, x)

    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model
   
def create_model():
    model=build_model()
    model = tf.keras.Model(model.input, model.layers[-2].output)
    outputs = tf.keras.layers.Dense(NUM_CLASSES,activation = "softmax", name = "output")(model.output)
    
    model = tf.keras.Model(model.input, outputs)
    return model

model=create_model()


##MOBILENET

In [ ]:
def create_model3():
    model = tf.keras.applications.MobileNetV3Large(weights= "imagenet",
                                    include_top=False,
                                    input_shape=(HEIGHT,WIDTH,CHANNELS), pooling="avg")
    predictions = tf.keras.layers.Dense(32, activation='sigmoid', name='predictions')(model.output)
    model = tf.keras.Model(inputs=model.input, outputs=predictions)
    
    # model.load_weights("../content/best_model.h5")
    model = tf.keras.Model(model.input, model.layers[-2].output)
    
    x = tf.keras.layers.Dense(512, activation = "relu")(model.output)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(256, activation = "relu")(x)
    # x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(128, activation = "relu")(x)
    x = tf.keras.layers.Dense(64, activation = "relu")(x)
    x = tf.keras.layers.Dense(64, activation = "relu")(x)
    outputs = tf.keras.layers.Dense(NUM_CLASSES, activation = "softmax", dtype = tf.float32)(x)
    model = tf.keras.Model(model.input,outputs)
    
    for layer in model.layers[:-14]:
        layer.trainble = False
    return model

model=create_model3()


##TFHUB MODEL

In [ ]:
import tensorflow_hub as hub


def create_model_hub():
    URL ='https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1'
    # 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'
    # "https://tfhub.dev/google/bit/m-r50x1/ilsvrc2012_classification/1"
    # "https://tfhub.dev/google/tf2-preview/inception_v3/classification/4"
    # 
    # https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
    # 'https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1' 

    feature_extractor = hub.KerasLayer(URL,input_shape = (HEIGHT,WIDTH,3))
    model=tf.keras.Sequential([feature_extractor])
    
    predictions = tf.keras.layers.Dense(16, activation='sigmoid', name='predictions')(model.output)
    model = tf.keras.Model(inputs=model.input, outputs=predictions)
    
    # model.load_weights("../content/best_model.h5")
    model = tf.keras.Model(model.input, model.layers[-2].output)
    
    x = tf.keras.layers.Dense(512, activation = "relu")(model.output)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(256, activation = "relu")(x)
    x = tf.keras.layers.Dense(64, activation = "relu")(x)
    # x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(64, activation = "relu")(x)
    outputs = tf.keras.layers.Dense(NUM_CLASSES, activation = "softmax", dtype = tf.float32)(x)
    model = tf.keras.Model(model.input,outputs)
    
    # for layer in model.layers[:-2]:
    #     layer.trainble = False
    return model

model=create_model_hub()


## RESNET

In [ ]:
def create_model4():
    model = tf.keras.applications.InceptionResNetV2(weights= "imagenet",
                                    include_top=False,
                                    input_shape=(HEIGHT,WIDTH,CHANNELS), pooling="avg")
    predictions = tf.keras.layers.Dense(12, activation='sigmoid', name='predictions')(model.output)
    model = tf.keras.Model(inputs=model.input, outputs=predictions)
    
    model = tf.keras.Model(model.input, model.layers[-2].output)
    
    x = tf.keras.layers.Dense(256, activation = "relu")(model.output)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(256, activation = "relu")(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(128, activation = "relu")(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(64, activation = "relu")(x)
    outputs = tf.keras.layers.Dense(NUM_CLASSES, activation = "softmax", dtype = tf.float32)(x)
    model = tf.keras.Model(model.input,outputs)
    
    # for layer in model.layers[:-1]:
    #     layer.trainble = False
    return model



model=create_model4()  

219070464/219055592 [==============================] - 1s 0us/step


## CUSTOM KERAS MODEL 

In [ ]:

IMAGE_SIZE=(224,224)
def conv_block(filters, inputs):
    x = tf.keras.layers.SeparableConv2D(filters, 3, activation="relu", padding="same")(inputs)
    x = tf.keras.layers.SeparableConv2D(filters, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    outputs = tf.keras.layers.MaxPool2D()(x)

    return outputs


def dense_block(units, dropout_rate, inputs):
    x = tf.keras.layers.Dense(units, activation="relu")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    outputs = tf.keras.layers.Dropout(dropout_rate)(x)

    return outputs
def build_model():
    inputs = tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    x = tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)
    x = tf.keras.layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = tf.keras.layers.MaxPool2D()(x)

    x = conv_block(32, x)
    x = conv_block(64, x)

    x = conv_block(128, x)
    x = tf.keras.layers.Dropout(0.2)(x)

    x = conv_block(256, x)
    x = tf.keras.layers.Dropout(0.2)(x)

    x = tf.keras.layers.Flatten()(x)
    x = dense_block(512, 0.7, x)
    x = dense_block(128, 0.5, x)
    x = dense_block(64, 0.3, x)

    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model
   
def create_model2():
    model=build_model()
    # model = tf.keras.models.load_model("../input/pneumonia-classification-challenge/xray_model.h5")
    model = tf.keras.Model(model.input, model.layers[-2].output)
    outputs = tf.keras.layers.Dense(6,activation = "softmax", name = "output")(model.output)
    
    model = tf.keras.Model(model.input, outputs)
    return model
# tf.keras.applications.DenseNet121
print("Keras Model")


Keras Model


## DENSENET121

In [ ]:
#best_score
def create_model():
    model = tf.keras.applications.DenseNet121(weights= "imagenet",
                                    include_top=False,
                                    input_shape=(HEIGHT,WIDTH,CHANNELS), pooling="avg")
    predictions = tf.keras.layers.Dense(32, activation='sigmoid', name='predictions')(model.output)
    model = tf.keras.Model(inputs=model.input, outputs=predictions)
    
    # model.load_weights("../content/best_model.h5")
    model = tf.keras.Model(model.input, model.layers[-2].output)
    
    x = tf.keras.layers.Dense(512, activation = "relu")(model.output)
    x = tf.keras.layers.Dropout(0.25)(x)
    x = tf.keras.layers.Dense(256, activation = "relu")(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(128, activation = "relu")(x)
    x = tf.keras.layers.Dense(64, activation = "relu")(x)
    x = tf.keras.layers.Dense(64, activation = "relu")(x)
    outputs = tf.keras.layers.Dense(NUM_CLASSES, activation = "softmax", dtype = tf.float32)(x)
    model = tf.keras.Model(model.input,outputs)
    
    for layer in model.layers[:-14]:
        layer.trainble = False
    return model

model = create_model()


29097984/29084464 [==============================] - 0s 0us/step


##LOADING SAVED MODEL

In [5]:
!wget https://huggingface.co/spaces/Sa-m/Brand-Logo-Classification/resolve/main/best_model.h5

--2022-09-19 14:40:48--  https://huggingface.co/spaces/Sa-m/Brand-Logo-Classification/resolve/main/best_model.h5
Resolving huggingface.co (huggingface.co)... 52.6.16.131, 52.202.207.64, 2600:1f18:147f:e850:db35:e0c7:187b:c770, ...
Connecting to huggingface.co (huggingface.co)|52.6.16.131|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/e5/9b/e59b7654cfdd413408f16738902b775e7655284994a5d5a63b80d7e6c692f6a9/7f963464697e2826ccaef798e4d98157700980f563bd5b3dbee2d27219be3499?response-content-disposition=attachment%3B%20filename%3D%22best_model.h5%22 [following]
--2022-09-19 14:40:49--  https://cdn-lfs.huggingface.co/repos/e5/9b/e59b7654cfdd413408f16738902b775e7655284994a5d5a63b80d7e6c692f6a9/7f963464697e2826ccaef798e4d98157700980f563bd5b3dbee2d27219be3499?response-content-disposition=attachment%3B%20filename%3D%22best_model.h5%22
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 13.227.254.47, 13.227.254.52, 13.227

In [18]:
from tensorflow.keras.models import load_model
model=load_model('/content/best_model.h5')

##COMPILING MODEL

In [ ]:
def compile_model(model, lr):
    
    optimizer =tf.keras.optimizers.Adam(learning_rate=lr)
        
    loss = "categorical_crossentropy"
    
    metrics = [
       tfa.metrics.F1Score(num_classes = NUM_CLASSES,average = "macro", name = "f1_score"),
       tf.keras.metrics.CategoricalAccuracy(name='acc')
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

In [ ]:
import datetime, os
%reload_ext tensorboard

METRIC = "val_f1_score"

def create_callbacks(metric = METRIC): 
    
    cpk_path = './best_model.h5'
    
    #Saving model checkpoints
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=cpk_path,
        monitor= metric,
        mode='max',
        save_best_only=True,
        verbose=1,
    )

    
    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    # Reducing learning rate
    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor= metric,
        mode='max',
        factor=0.1,
        patience=1,
        verbose=1
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor= metric,
        mode='max',
        patience=10, 
        verbose=1
    )
    
    callbacks = [checkpoint, reducelr, earlystop,tensorboard_callback]         
    
    return callbacks

## MODEL EVALUATION

In [ ]:
EPOCHS= 100
VERBOSE =1

tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
  
  
  model = compile_model(model, lr=0.0001)

  callbacks = create_callbacks()

  history = model.fit(train_ds, 
                      epochs=EPOCHS,
                      callbacks=callbacks,
                      validation_data = val_ds,
                        verbose=VERBOSE)

Epoch 1/100
170/170 [==============================] - ETA: 0s - loss: 0.9066 - f1_score: 0.4283 - acc: 0.6730
Epoch 1: val_f1_score improved from -inf to 0.73859, saving model to ./best_model.h5
170/170 [==============================] - 59s 207ms/step - loss: 0.9066 - f1_score: 0.4283 - acc: 0.6730 - val_loss: 0.3829 - val_f1_score: 0.7386 - val_acc: 0.9132 - lr: 1.0000e-04
Epoch 2/100
170/170 [==============================] - ETA: 0s - loss: 0.2721 - f1_score: 0.8608 - acc: 0.9271
Epoch 2: val_f1_score improved from 0.73859 to 0.87614, saving model to ./best_model.h5
170/170 [==============================] - 30s 175ms/step - loss: 0.2721 - f1_score: 0.8608 - acc: 0.9271 - val_loss: 0.1598 - val_f1_score: 0.8761 - val_acc: 0.9474 - lr: 1.0000e-04
Epoch 3/100
170/170 [==============================] - ETA: 0s - loss: 0.0988 - f1_score: 0.9592 - acc: 0.9764
Epoch 3: val_f1_score did not improve from 0.87614

Epoch 3: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.


##PREDICTING TEST DATA

In [19]:

pred = model.predict(test_ds)
labels = (train_ds.class_indices)
predicted_class_indices=np.argmax(pred,axis=1)
labels = dict((v,k) for k,v in labels.items())

predictedLables= [labels[k] for k in predicted_class_indices]

actualLables= [labels[k] for k in test_ds.classes]


In [ ]:
accuracy_score(actualLables,predictedLables)

0.7660714285714286

In [21]:
filenames=test_ds.filenames

directory= test_ds.directory
results=pd.DataFrame({"Actual": actualLables,
                      "Predictions":predictedLables,
                      "Directory":directory
                      })
# results=pd.DataFrame(predictedLables)
results.to_csv("results.txt",index=False,header=False)

In [22]:
results.head()

,Actual,Predictions,Directory
0,Burger King,McDonalds,../content/logos3/test
1,Burger King,McDonalds,../content/logos3/test
2,Burger King,McDonalds,../content/logos3/test
3,Burger King,McDonalds,../content/logos3/test
4,Burger King,Burger King,../content/logos3/test


## Test data from Video

In [ ]:

pred = model.predict(my_test)
labels = (train_ds.class_indices)
predicted_class_indices=np.argmax(pred,axis=1)
labels = dict((v,k) for k,v in labels.items())

predictedLables= [labels[k] for k in predicted_class_indices]

In [ ]:

results=pd.DataFrame(predictedLables)
results.to_csv("results.txt",index=False,header=False)

## Download Model

In [ ]:
from google.colab import files
files.download('/content/best_model.h5')